In [22]:
import csv
import math
import random
import numpy as np
import pandas as pd

In [23]:
#Preparing Data
train_data = pd.read_excel("Train_Data_Univ.xlsx")
test_data = pd.read_excel("Test_Data_Univ.xlsx")

X_train_data = train_data.drop(["University Name","University State","Private/Public"],axis=1)
Y_train_data = train_data["Private/Public"]


X_test_data = test_data.drop(["University Name","University State","Private/Public"],axis=1)
Y_test_data = test_data["Private/Public"]

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Y_train_data = le.fit_transform(Y_train_data)
Y_test_data = le.fit_transform(Y_test_data)


X_train_data["Private/Public"]  = Y_train_data
X_test_data["Private/Public"] = Y_test_data
X_train_data = X_train_data.values
X_test_data = X_test_data.values

In [24]:
#Class Separation
def separteByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [25]:
#Mean Calculation
def mean(numbers):
    return sum(numbers)/float(len(numbers))

In [26]:
#SD Calculation
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

In [27]:
#Data Summarizing
def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

In [28]:
#Summarizing by Class
def summarizeByClass(dataset):
    separated = separteByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

In [29]:
#Calculating Probability
def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return(1/(math.sqrt(2*math.pi)*stdev))*exponent

In [30]:
#Calculation Probability by class
def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
        return probabilities

In [31]:
#Predicting
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [32]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [33]:
#Calculating Accuracy
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet)))*100.0

In [34]:
#Calculating Confusion Matrix
def perf_measure(y_actual, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_pred)): 
        if y_actual[i][-1]==y_pred[i]==1:
            TP += 1
        if y_pred[i]==1 and y_actual[i][-1]!=y_pred[i]:
            FP += 1
        if y_actual[i][-1]==y_pred[i]==0:
            TN += 1
        if y_pred[i]==0 and y_actual[i][-1]!=y_pred[i]:
            FN += 1
    return(TP, FP,TN, FN)


In [37]:
def main():
    dataset = X_train_data
    trainingSet = X_train_data
    testSet = X_test_data 
    summaries = summarizeByClass(trainingSet)
    predictions =  getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    confusion_matrix = perf_measure(testSet, predictions)
    print('Accuracy: {0}%' .format(accuracy))
    print("confustion matrix(TP, FP,TN, FN): {}".format(confusion_matrix))
main()

Accuracy: 60.0%
confustion matrix(TP, FP,TN, FN): (0, 0, 3, 2)
